## [Day 8](https://adventofcode.com/2020/day/8)

Here we have a bunch of instructions for a computer where they take three different values:
1. An `acc` which adds the argument to another variable called the accumulator
2. A `jmp` which jumps the instructions that many lines
2. A `nop` which does nothing

We're asked to find an infinite loop and what the value of the accelerator is right before that loop restarts.

In [54]:
import pandas as pd
import numpy as np

inst = open('../inputs/d8.txt').read().splitlines()

inst = [x.replace('+', '').split(' ', maxsplit = 2) for x in inst]
inst = pd.DataFrame(inst)
inst.columns = ['opp', 'arg']
inst['arg'] = inst['arg'].astype('int64')
inst['reached'] = np.nan
inst.head()

,opp,arg,reached
0,acc,42,NaN
1,acc,-12,NaN
2,nop,112,NaN
3,acc,47,NaN
4,jmp,600,NaN


So I feel like we can just run a while loop and mark the ones that have been executed?

In [55]:
accu = 0
i = 0
while True:
    if inst.reached[[i]].isna()[i]:
        inst.loc[i,'reached'] = 1
        opp = inst.opp[i]

        if opp == 'nop':
            i += 1
        elif opp == 'jmp':
            i += inst.arg[i]
        else:
            accu += inst.arg[i]
            i += 1
    else:
        print(accu)
        break

1859


K  that was easy...

### Part 2

Ok so we're told now that there is one `jmp` or `nop` that is supposed to be flipped to the other in order for this to work. I think that there is probably some super smart way to figure it out but I don't see it in the few moments I've been thinking about this. I think we could make a list of candidates to shorten the search and then perform the same kinda process.

In [56]:
inst.reached.value_counts()

1.0    213
Name: reached, dtype: int64

In [58]:
inst.shape

(683, 3)

In [60]:
candidates = inst.query('reached == 1 and opp in ("nop", "jmp")')
candidates = candidates.query('arg != 1 or opp == "nop"')
candidates.head()

,opp,arg,reached
2,nop,112,1.0
4,jmp,600,1.0
11,jmp,56,1.0
17,nop,580,1.0
19,jmp,112,1.0


In [62]:
candidates.shape

(91, 3)

Okay so that is a smaller set of values to try and change

In [66]:
def test_index(ind):
    
    # Make a copy and change the one value
    test_frame = inst.copy()
    test_frame['reached'] = np.nan
    if test_frame.loc[ind,'opp'] == 'jmp':
        test_frame.loc[ind,'opp'] = 'nop'
    else:
        test_frame.loc[ind,'opp'] = 'jmp'
    
    # Now run the loop:
    accu = 0
    i = 0
    while True:
        # Add this piece where
        if i == test_frame.shape[0]:
            print(accu)
            return(1)
        if test_frame.reached[[i]].isna()[i]:
            test_frame.loc[i,'reached'] = 1
            opp = test_frame.opp[i]

            if opp == 'nop':
                i += 1
            elif opp == 'jmp':
                i += test_frame.arg[i]
            else:
                accu += test_frame.arg[i]
                i += 1
        else:
            return(0)
            
for ind in candidates.index:
    if test_index(ind) == 1:
        break

1235


Welllllll that took about 15 minutes total... and the previous day took me like 4 hours lol...